# Módulo 3 - Modelado

## Objetivos

## Librerías

In [1]:

from sklearn.model_selection import train_test_split as sklearn_train_test_split


from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import numpy as np
import pandas as pd

import kagglehub

/home/druiz35/Documents/SEpsilon/SEpsilon-Proyectos/AI-ML-Data/RNABI2025-1-Equipo3/Trabajo3/.surprise_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cargado de datos

In [2]:
# Download latest version
general_path = kagglehub.dataset_download("amanmehra23/travel-recommendation-dataset")

print("Path to dataset files:", general_path)

Path to dataset files: /home/druiz35/.cache/kagglehub/datasets/amanmehra23/travel-recommendation-dataset/versions/1


In [3]:
destinations_path = general_path + "/Expanded_Destinations.csv"
destinations_df = pd.read_csv(destinations_path)

reviews_path = general_path + "/Final_Updated_Expanded_Reviews.csv"
reviews_df = pd.read_csv(reviews_path)

userhistory_path = general_path + "/Final_Updated_Expanded_UserHistory.csv"
userhistory_df = pd.read_csv(userhistory_path)

users_path = general_path + "/Final_Updated_Expanded_Users.csv"
users_df = pd.read_csv(users_path)

dataframes = {
    "Destinations": destinations_df,
    "Reviews": reviews_df,
    "User History": userhistory_df,
    "Users":users_df
}

In [4]:
path = "m3_merged_df.csv"
df = pd.read_csv(path)
pd.set_option('display.max_columns', None)
df

,Unnamed: 0,ReviewID,DestinationID_x,UserID,Rating,ReviewText,Name_x,State,Type,Popularity,BestTimeToVisit,HistoryID,DestinationID_y,VisitDate,ExperienceRating,Name_y,Email,Preferences,Gender,NumberOfAdults,NumberOfChildren
0,0,1,178,327,2,Incredible monument!,Jaipur City,Rajasthan,City,8.544352,Oct-Mar,79,175,2024-01-01,3,Pooja,pooja@example.com,"City, Historical",Female,1,1
1,1,2,411,783,1,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,8.284127,Nov-Feb,834,894,2024-03-20,2,Karan,karan@example.com,"City, Historical",Male,1,1
2,2,4,358,959,3,Incredible monument!,Jaipur City,Rajasthan,City,7.738761,Oct-Mar,998,660,2024-02-15,4,Ritvik,ritvik@example.com,"Nature, Adventure",Male,1,1
3,3,5,989,353,2,Loved the beaches!,Kerala Backwaters,Kerala,Nature,8.208088,Sep-Mar,202,894,2024-01-01,5,Isha,isha@example.com,"Nature, Adventure",Female,2,0
4,4,6,473,408,4,A historical wonder,Jaipur City,Rajasthan,City,8.138558,Oct-Mar,331,403,2024-01-01,2,Ishaan,ishaan@example.com,"City, Historical",Male,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,988,991,701,850,3,Incredible monument!,Taj Mahal,Uttar Pradesh,Historical,8.814029,Nov-Feb,138,131,2024-03-20,1,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,0
989,989,991,701,850,3,Incredible monument!,Taj Mahal,Uttar Pradesh,Historical,8.814029,Nov-Feb,643,761,2024-01-01,4,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,0
990,990,995,231,346,5,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,7.788256,Nov-Feb,454,113,2024-01-01,2,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,2
991,991,995,231,346,5,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,7.788256,Nov-Feb,556,128,2024-01-01,4,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,2


In [5]:
df = df.drop(df.columns[[0,1]], axis=1)
df

,DestinationID_x,UserID,Rating,ReviewText,Name_x,State,Type,Popularity,BestTimeToVisit,HistoryID,DestinationID_y,VisitDate,ExperienceRating,Name_y,Email,Preferences,Gender,NumberOfAdults,NumberOfChildren
0,178,327,2,Incredible monument!,Jaipur City,Rajasthan,City,8.544352,Oct-Mar,79,175,2024-01-01,3,Pooja,pooja@example.com,"City, Historical",Female,1,1
1,411,783,1,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,8.284127,Nov-Feb,834,894,2024-03-20,2,Karan,karan@example.com,"City, Historical",Male,1,1
2,358,959,3,Incredible monument!,Jaipur City,Rajasthan,City,7.738761,Oct-Mar,998,660,2024-02-15,4,Ritvik,ritvik@example.com,"Nature, Adventure",Male,1,1
3,989,353,2,Loved the beaches!,Kerala Backwaters,Kerala,Nature,8.208088,Sep-Mar,202,894,2024-01-01,5,Isha,isha@example.com,"Nature, Adventure",Female,2,0
4,473,408,4,A historical wonder,Jaipur City,Rajasthan,City,8.138558,Oct-Mar,331,403,2024-01-01,2,Ishaan,ishaan@example.com,"City, Historical",Male,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,701,850,3,Incredible monument!,Taj Mahal,Uttar Pradesh,Historical,8.814029,Nov-Feb,138,131,2024-03-20,1,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,0
989,701,850,3,Incredible monument!,Taj Mahal,Uttar Pradesh,Historical,8.814029,Nov-Feb,643,761,2024-01-01,4,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,0
990,231,346,5,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,7.788256,Nov-Feb,454,113,2024-01-01,2,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,2
991,231,346,5,Loved the beaches!,Taj Mahal,Uttar Pradesh,Historical,7.788256,Nov-Feb,556,128,2024-01-01,4,Hitesh,hitesh@example.com,"Beaches, Historical",Male,2,2


## Similarity Score Filtering

In [6]:
df["description"] = df['Type'] + ' ' + df['State'] + ' ' + df['BestTimeToVisit'] + " " + df['Preferences']

In [7]:
vectorizer = CountVectorizer(stop_words='english')
#vectorizer = TfidfVectorizer(stop_words='english')
destination_features = vectorizer.fit_transform(df['description'])


In [8]:
cosine_sim = cosine_similarity(destination_features, destination_features)
cosine_sim

array([[1.        , 0.47140452, 0.72168784, ..., 0.23570226, 0.23570226,
        1.        ],
       [0.47140452, 1.        , 0.13608276, ..., 0.88888889, 0.88888889,
        0.47140452],
       [0.72168784, 0.13608276, 1.        , ..., 0.        , 0.        ,
        0.72168784],
       ...,
       [0.23570226, 0.88888889, 0.        , ..., 1.        , 1.        ,
        0.23570226],
       [0.23570226, 0.88888889, 0.        , ..., 1.        , 1.        ,
        0.23570226],
       [1.        , 0.47140452, 0.72168784, ..., 0.23570226, 0.23570226,
        1.        ]])

In [9]:
user_item_matrix = userhistory_df.pivot(index='UserID', columns='DestinationID', values='ExperienceRating')
user_item_matrix = user_item_matrix.fillna(0)
user_item_matrix


DestinationID,2,3,4,5,6,7,8,9,10,11,14,16,17,19,20,21,22,24,25,27,29,30,33,35,36,37,38,39,40,41,42,48,49,50,51,53,55,58,59,61,62,64,65,66,67,68,71,75,76,78,79,80,81,82,83,84,85,86,87,89,90,91,92,94,96,97,100,101,102,103,105,106,108,109,110,112,113,114,115,117,118,119,120,122,126,127,128,130,131,133,134,135,137,139,142,145,146,148,149,152,153,154,158,159,160,161,164,166,167,168,169,170,171,174,175,176,177,180,181,182,185,186,188,189,190,191,192,194,195,196,198,199,200,201,202,203,204,205,208,209,211,216,217,218,220,221,222,223,224,227,228,229,230,235,236,239,240,242,245,250,251,253,254,258,259,261,262,263,265,266,268,269,270,271,272,273,274,277,278,280,281,282,284,285,287,288,289,290,291,292,294,295,296,297,301,303,305,306,307,310,311,315,316,317,319,320,322,324,326,327,329,330,331,333,335,336,339,340,344,346,347,348,349,350,352,353,354,355,356,357,359,360,361,363,365,366,367,368,369,371,372,373,374,377,378,380,381,383,384,385,386,387,388,389,390,391,392,393,394,395,396,398,399,401,402,403,404,405,408,409,410,411,413,414,416,417,419,420,421,422,423,424,425,428,429,430,431,432,433,434,435,436,437,441,443,445,447,448,451,452,457,458,459,460,463,466,467,468,469,470,471,472,474,476,477,478,479,480,481,482,483,484,485,487,488,489,490,491,493,494,495,496,499,500,501,504,505,507,509,510,511,513,518,519,520,523,525,528,529,531,532,535,537,539,541,544,545,546,547,549,552,554,555,556,559,562,565,566,567,568,569,572,573,575,576,578,581,582,583,584,586,587,589,590,592,597,601,606,610,611,612,613,614,617,618,619,621,622,624,627,628,631,632,633,635,636,637,638,641,642,643,645,646,647,648,650,653,654,655,657,659,660,661,662,667,670,674,675,676,677,678,679,682,683,684,685,688,689,690,691,692,694,696,697,698,699,700,701,702,705,706,707,709,711,712,714,715,717,718,719,720,721,723,724,725,726,727,728,730,731,732,734,735,737,739,747,748,749,750,752,753,754,756,757,760,761,763,764,766,767,768,770,771,772,773,774,775,776,779,782,784,785,786,788,791,793,795,796,797,799,800,801,803,804,805,806,807,808,811,813,814,818,821,823,824,825,826,827,828,830,831,833,834,835,836,837,839,840,847,848,849,850,852,853,859,864,865,866,867,869,871,872,873,874,877,878,879,880,881,883,884,886,889,890,892,893,894,897,898,899,901,903,904,905,908,909,910,911,912,913,916,920,921,922,925,926,928,931,932,933,935,936,938,939,941,944,945,946,948,950,951,952,953,954,955,956,957,958,959,960,962,964,965,966,967,968,969,970,972,974,975,977,978,979,983,984,985,986,987,988,990,991,993,994,996,997,998,1000
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [10]:
user_similarity = cosine_similarity(user_item_matrix)
user_similarity


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [11]:
def collaborative_recommend(user_id, user_similarity, user_item_matrix, destinations_df):
    similar_users = user_similarity[user_id - 1]
    similar_users_idx = np.argsort(similar_users)[::-1][1:6]
    similar_user_ratings = user_item_matrix.iloc[similar_users_idx].mean(axis=0)
    recommended_destinations_ids = similar_user_ratings.sort_values(ascending=False).head(5).index
    recommendations = destinations_df[destinations_df['DestinationID'].isin(recommended_destinations_ids)][[
        'DestinationID', 'Name', 'State', 'Type', 'Popularity', 'BestTimeToVisit'
    ]]
    return recommendations

In [12]:

collaborative_recommendations = collaborative_recommend(10, user_similarity, user_item_matrix, destinations_df)
collaborative_recommendations

,DestinationID,Name,State,Type,Popularity,BestTimeToVisit
519,520,Leh Ladakh,Jammu and Kashmir,Adventure,8.521876,Apr-Jun
524,525,Leh Ladakh,Jammu and Kashmir,Adventure,8.090624,Apr-Jun
684,685,Leh Ladakh,Jammu and Kashmir,Adventure,7.598827,Apr-Jun
755,756,Taj Mahal,Uttar Pradesh,Historical,8.547608,Nov-Feb
784,785,Leh Ladakh,Jammu and Kashmir,Adventure,9.366933,Apr-Jun


In [13]:
collaborative_recommendations = collaborative_recommend(11, user_similarity, user_item_matrix, destinations_df)
collaborative_recommendations


,DestinationID,Name,State,Type,Popularity,BestTimeToVisit
5,6,Taj Mahal,Uttar Pradesh,Historical,7.648950,Nov-Feb
367,368,Jaipur City,Rajasthan,City,8.008252,Oct-Mar
519,520,Leh Ladakh,Jammu and Kashmir,Adventure,8.521876,Apr-Jun
784,785,Leh Ladakh,Jammu and Kashmir,Adventure,9.366933,Apr-Jun
969,970,Leh Ladakh,Jammu and Kashmir,Adventure,9.013101,Apr-Jun


## Testing

In [14]:
def split_user_histories(userhistory_df, test_size=0.2):
    train_list = []
    test_list = []

    for user_id, group in userhistory_df.groupby("UserID"):
        if len(group) < 2:
            continue 
        train, test = sklearn_train_test_split(group, test_size=test_size, random_state=42)
        train_list.append(train)
        test_list.append(test)

    return pd.concat(train_list), pd.concat(test_list)

train_history, test_history = split_user_histories(userhistory_df)

In [15]:
train_history

,HistoryID,UserID,DestinationID,VisitDate,ExperienceRating
598,599,1,941,2024-02-15,5
30,31,2,385,2024-01-01,5
480,481,3,904,2024-01-01,4
854,855,7,670,2024-03-20,1
392,393,8,50,2024-03-20,5
...,...,...,...,...,...
930,931,978,76,2024-01-01,4
4,5,989,389,2024-02-15,4
515,516,992,296,2024-03-20,5
780,781,992,259,2024-01-01,1


In [16]:
test_history

,HistoryID,UserID,DestinationID,VisitDate,ExperienceRating
875,876,1,467,2024-03-20,3
450,451,2,437,2024-01-01,2
543,544,3,421,2024-01-01,4
903,904,7,953,2024-01-01,5
407,408,8,204,2024-03-20,5
...,...,...,...,...,...
245,246,976,683,2024-03-20,3
920,921,978,547,2024-03-20,2
11,12,989,779,2024-03-20,4
351,352,992,62,2024-01-01,4


In [28]:
def evaluate_precision_at_k_collab(user_item_matrix, user_similarity, destinations_df,
                                   train_history, test_history, k=5):
    precisions = []
    user_ids = train_history['UserID'].unique()

    for user_id in user_ids:
        true_destinations = set(test_history[test_history['UserID'] == user_id]['DestinationID'])
        if len(true_destinations) == 0:
            continue 
        try:
            recommendations = collaborative_recommend(
                user_id=user_id,
                user_similarity=user_similarity,
                user_item_matrix=user_item_matrix,
                destinations_df=destinations_df
            )
        except IndexError:
            continue 

        recommended_ids = set(recommendations['DestinationID'].values[:k])
        hits = recommended_ids & true_destinations
        print(recommended_ids, true_destinations)

        precision = len(hits) / k
        precisions.append(precision)

    avg_precision = np.mean(precisions) if precisions else 0.0
    print(f"Average Precision@{k}: {avg_precision:.4f}")
    return avg_precision



In [29]:
evaluate_precision_at_k_collab(
    user_item_matrix=user_item_matrix,
    user_similarity=user_similarity,
    destinations_df=destinations_df,
    train_history=train_history,
    test_history=test_history,
    k=5
)


{520, 685, 879, 336, 785} {467}
{520, 525, 685, 785, 756} {437}
{320, 421, 38, 872, 661} {421}
{899, 996, 811, 912, 959} {953}
{520, 525, 685, 785, 756} {204}
{811, 204, 175, 50, 340} {959}
{520, 11, 811, 685, 785} {994}
{520, 11, 811, 685, 785} {222}
{811, 11, 525, 756, 725} {970}
{520, 11, 811, 685, 785} {764}
{227, 100, 811, 952, 667} {582}
{6, 520, 685, 785, 436} {674}
{835, 653, 303, 468, 189} {702}
{675, 38, 583, 938, 723} {886}
{864, 353, 525, 685, 756} {91}
{520, 685, 494, 785, 986} {500}
{4, 170, 685, 20, 92} {670}
{11, 108, 525, 685, 756} {474}
{480, 811, 525, 685, 756} {21}
{805, 229, 8, 75, 811} {808}
{520, 653, 303, 785, 477} {421}
{67, 811, 11, 685, 723} {747}
{520, 811, 685, 785, 436} {494}
{289, 11, 811, 984, 537} {835}
{811, 11, 685, 723, 828} {170}
{485, 394, 11, 685, 532} {29}
{520, 685, 366, 785, 531} {581}
{520, 685, 624, 785, 723} {873}
{38, 583, 586, 685, 723} {254}
{967, 552, 839, 525, 756} {383}
{583, 685, 208, 723, 347} {954}
{811, 11, 685, 723, 347} {101}
{93

0.004733727810650888